In [2]:
from starknet_py.hash.address import compute_address
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.signer.stark_curve_signer import KeyPair
from starknet_py.net.models.chains import StarknetChainId
from ast import literal_eval  


# You can also generate a key pair
key_pair_generated = KeyPair.generate()


# Prefund the address (using the token bridge or by sending fee tokens to the computed address)
# Make sure the tx has been accepted on L2 before proceeding

# Define the client to be used to interact with Starknet
client = FullNodeClient(node_url="https://starknet-sepolia.infura.io/v3/1bc8c51edac6463eaebc92a80761f076")
class_hash: int = literal_eval("0x05400e90f7e0ae78bd02c77cd75527280470e2fe19c54970dd79dc37a9d3645c")
salt: int = 456

account = Account(
    client=client,
    address=compute_address(
        salt=salt,
        class_hash= class_hash,  # class_hash of the Account declared on the Starknet
        constructor_calldata=[key_pair_generated.public_key],
        deployer_address=0,
    ),
    key_pair=key_pair_generated,
    chain=StarknetChainId.SEPOLIA,
)

print("Address: ", hex(account.address))

Address:  0x661a8612b6d3b62eebe3ece57535401c9008c2aaaa64c2e92b957b2451b6a8f


Deploy account

In [10]:

print(await client.get_block_with_txs(block_number=294444))
# Use `Account.deploy_account_v1` or `Account.deploy_account_v3` static methods to deploy an account
# account_deployment_result = await Account.deploy_account_v1(
#     address=account.address,
#     class_hash=class_hash,
#     salt=salt,
#     key_pair=key_pair_generated,
#     client=client,
#     constructor_calldata=[key_pair_generated.public_key],
#     max_fee=int(1)
# )

# account_deployment_result.__dict__


ValidationError: {'l1_data_gas_price': ['Missing data for required field.'], 'l1_da_mode': ['Missing data for required field.']}

Wait for account acceptance

In [93]:
# Wait for deployment transaction to be accepted
await account_deployment_result.wait_for_acceptance()

# From now on, account can be used as usual
account_result = account_deployment_result.account

account_result

ValidationError: {'execution_resources': {'data_availability': ['Missing data for required field.']}}

Sample interaction

In [11]:
from starknet_py.net.full_node_client import FullNodeClient


call_result = await client.get_block()
call_result

ValidationError: {'l1_data_gas_price': ['Missing data for required field.'], 'l1_da_mode': ['Missing data for required field.']}

Deploy contract

In [ ]:
from starknet_py.contract import Contract
from starknet_py.net.client_models import ResourceBounds

l1_resource_bounds = ResourceBounds(
    max_amount=int(1e5), max_price_per_unit=int(1e13)
)
# Declare and deploy an example contract which implements a simple k-v store.
# Contract.declare_v3 takes string containing a compiled contract (sierra) and
# a class hash (casm_class_hash) or string containing a compiled contract (casm)
declare_result = await Contract.declare_v3(
    account,
    compiled_contract=compiled_contract,
    compiled_class_hash=class_hash,
    l1_resource_bounds=l1_resource_bounds,
)

await declare_result.wait_for_acceptance()
deploy_result = await declare_result.deploy_v3(
    l1_resource_bounds=l1_resource_bounds,
)
# Wait until deployment transaction is accepted
await deploy_result.wait_for_acceptance()

# Get deployed contract
map_contract = deploy_result.deployed_contract
k, v = 13, 4324
# Adds a transaction to mutate the state of k-v store. The call goes through account proxy, because we've used
# Account to create the contract object
await (
    await map_contract.functions["put"].invoke_v3(
        k,
        v,
        l1_resource_bounds=ResourceBounds(
            max_amount=int(1e5), max_price_per_unit=int(1e13)
        ),
    )
).wait_for_acceptance()

# Retrieves the value, which is equal to 4324 in this case
(resp,) = await map_contract.functions["get"].call(k)

# There is a possibility of invoking the multicall

# Creates a list of prepared function calls
calls = [
    map_contract.functions["put"].prepare_invoke_v3(key=10, value=20),
    map_contract.functions["put"].prepare_invoke_v3(key=30, value=40),
]

# Executes only one transaction with prepared calls
transaction_response = await account.execute_v3(
    calls=calls,
    l1_resource_bounds=l1_resource_bounds,
)
await account.client.wait_for_tx(transaction_response.transaction_hash)